In [ ]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.5.0.jar

--2022-10-04 01:53:49--  https://jdbc.postgresql.org/download/postgresql-42.5.0.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1046274 (1022K) [application/java-archive]
Saving to: ‘postgresql-42.5.0.jar’

postgresql-42.5.0.j 100%[===================>]   1022K  6.45MB/s    in 0.2s    

2022-10-04 01:53:50 (6.45 MB/s) - ‘postgresql-42.5.0.jar’ saved [1046274/1046274]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.5.0.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [ ]:
from pyspark import SparkFiles
url = "https://dilnigardolkun.s3.us-west-2.amazonaws.com/amazon_reviews_us_Video_v1_00.tsv"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Video_v1_00.tsv"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   49033728|R1P1G5KZ05H6RD|6302503213|     748506413|The Night They Sa...|           Video|          5|            0|          0|   N|                Y|    Very satisfied!!|Fast shipping. Pl...| 2015-08-31|
|         US|   17857748|R106N066IUN8ZV|B000059PET|     478710180|Hamlet / Kline, N...|           Video|          5|    

### Create DataFrames to match tables

In [ ]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame


In [ ]:
# Create the customers_table DataFrame
customers_df = df.groupby("customer_id").agg({'customer_id': 'count'}).withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   35645596|             1|
|   50851028|             1|
|   20686161|             2|
|   15191478|             2|
|    2202715|             1|
|   48333068|             1|
|   22910908|             2|
|   38463562|             1|
|   26902200|             1|
|   10546543|             1|
|   33236947|             1|
|   40127650|             1|
|   41244127|             3|
|   22968117|             1|
|   25472255|             1|
|    3160356|             1|
|   18785199|             2|
|   19840529|             1|
|    1655605|             1|
|   14676950|             3|
+-----------+--------------+
only showing top 20 rows



In [ ]:
# Create the products_table DataFrame and drop duplicates. 
products_df = df.select(["product_id", "product_title"]).drop_duplicates(["product_id"])
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|0001485423|The Life of Siste...|
|0001499572|At Home with the ...|
|0001501348|Lift Him Up With ...|
|0001526863|Steve Green: Hide...|
|0001527665|   Peace Child [VHS]|
|0005000009|Where Jesus Walke...|
|0005019281|An American Chris...|
|0005022290|Live! The Young M...|
|0005041104|Camp Harmony (Rob...|
|0005048524|Dinosaurs and the...|
|0005092663|A NATION ADRIFT  ...|
|0005114349|  Christmas is [VHS]|
|0005119359|         Jacob [VHS]|
|0005119367|        Joseph [VHS]|
|0005165687|Laugh Yo' Self 2 ...|
|0005166004|The Bible Collect...|
|0005200288|Walk Where Jesus ...|
|0005253500|The Donut Man: Je...|
|0005257859|Passion for His P...|
|0005257891|Breaking Intimida...|
+----------+--------------------+
only showing top 20 rows



In [ ]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
review_id_df = df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1P1G5KZ05H6RD|   49033728|6302503213|     748506413| 2015-08-31|
|R106N066IUN8ZV|   17857748|B000059PET|     478710180| 2015-08-31|
| R7WTAA1S5O7D9|   25551507|0788812807|     981002815| 2015-08-31|
|R32HFMVWLYOYJK|   21025041|6302509939|     333219811| 2015-08-31|
| RWT3H6HBVAL6G|   40943563|B00JENS2BI|     538101194| 2015-08-31|
|R1S3T3GWUGQTW7|   17013969|6305761302|     716303278| 2015-08-31|
|R3R0QYHA800REE|   47611685|6300157555|     134996462| 2015-08-31|
|R1FR0EQPHPW5NM|   35680737|6300189570|     498116870| 2015-08-31|
| RGORN81H45NI7|   10747909|B000SXQ5US|      77519275| 2015-08-31|
|R1CNYN4ABNOJSU|     126341|B00008F22G|     917778300| 2015-08-31|
|R2DW06821PMN40|   40676812|6303453961|     187850980| 2015-08-31|
|R1CS8AMA8B0VBJ|   19863533|6302453178|     901056605| 2015-08

In [ ]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1P1G5KZ05H6RD|          5|            0|          0|   N|                Y|
|R106N066IUN8ZV|          5|            0|          0|   N|                Y|
| R7WTAA1S5O7D9|          4|            0|          0|   N|                Y|
|R32HFMVWLYOYJK|          5|            0|          0|   N|                Y|
| RWT3H6HBVAL6G|          3|            0|          0|   N|                N|
|R1S3T3GWUGQTW7|          5|            0|          0|   N|                Y|
|R3R0QYHA800REE|          4|            0|          0|   N|                Y|
|R1FR0EQPHPW5NM|          4|            1|          2|   N|                N|
| RGORN81H45NI7|          5|            1|          1|   N|                Y|
|R1CNYN4ABNOJSU|          5|            0|          0|   N|     

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [ ]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://dataclass.cxwyzoqepck1.us-west-2.rds.amazonaws.com:5432/postgres"
config = {"user":"postgres", 
          "password": "password", 
          "driver":"org.postgresql.Driver"}
# REMOVED PASSWORD FOR SECURITY PURPOSE

In [ ]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [ ]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [ ]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)